# Add Genome, Gene, and Protein info for Novel Coronavirus (COVID-19)¶
## **WORK IN PROGRESS**

In [1]:
import os
from py2neo import Graph

### Open Graph data base

In [2]:
graph = Graph(password="neo4jbinder")

Remove any existing nodes and relationships

In [3]:
#graph.run("MATCH (n) DETACH DELETE n")

Get absolute path for data directory (LOAD CSV requires absolute path)

In [4]:
data_dir = os.path.abspath("../data")

Make sure that the core entities are unique

In [5]:
graph.run("CREATE CONSTRAINT ON (ge:Genome) ASSERT ge.name IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (g:Gene) ASSERT g.name IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (p:Protein) ASSERT p.name IS UNIQUE")

### Add genome info

In [6]:
load_genome = f"LOAD CSV WITH HEADERS FROM 'file:///{data_dir}/genome_COVID-19.csv' AS e"
query = """
MERGE (ge:Genome {name:e.name})
SET ge.accession = e.accession,
    ge.organism = e.organism,
    ge.taxon = e.taxon,
    ge.isolate = e.isolate,
    ge.host = e.host,
    ge.start = e.start,
    ge.end = e.end
MERGE (p: Pathogen{name: 'Wuhan seafood market pneumonia virus'})
MERGE (ge)-[:ENCODES]->(p)
"""
graph.run(load_genome + query)

### Add gene and protein info

In [7]:
load_gene = f"LOAD CSV WITH HEADERS FROM 'file:///{data_dir}/genes_COVID-19.csv' AS e"
query = """
MERGE (g:Gene {name:e.name})
SET g.gene_id = e.gene_id,
    g.start = e.start,
    g.end = e.end
MERGE (ge:Genome {name:'Wuhan seafood market pneumonia virus isolate Wuhan-Hu-1'})
MERGE (p:Protein {name:e.protein_name})
SET p.protein_id = e.protein_id
MERGE (ge)-[:CONTAINS]->(g)
MERGE (g)-[:ENCODES]->(p)
"""
graph.run(load_gene + query)